In [1]:
from typing import Any

import torch
from torch import Tensor
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.nn import MessagePassing
from torch_geometric.datasets import Planetoid

In [2]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add', flow='source_to_target')  # 'Add' aggregation. 消息冲源节点到目标节点
        self.linear = torch.nn.Linear(in_channels, out_channels)
    
    def forward(self, x, edge_index) -> Any:
        """
        :param x in shape of [N, in_channels] 
        :param edge_index in shape of [2, E]
        :return: 
        """
        # Step-1: Add self-loops to the adjacency matrix
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        # Step-2: Linearly transform node feature matrix
        x = self.linear(x)
        # Step-3: Compute normalization
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        # Step-4, Step-5: Start propagating messages
        return self.propagate(edge_index, x=x, norm=norm, deg=deg.view(-1, 1))
    
    def message(self, x_j, norm, deg_i) -> Tensor:
        """
        :param x_j in shape of [E, out_channels]
        :param norm: 
        :return: 
        """
        # Step-4: Normalize node features
        return norm.view(-1, 1) * x_j * deg_i
        

In [28]:
dataset = Planetoid(root='../datasets/Planetoid', name='Cora')
data = dataset[0]
net = GCNConv(data.num_features, 64)
h_nodes = net(data.x, data.edge_index)
print(h_nodes.shape)



torch.Size([2708, 64])


In [19]:
deg = degree(data.edge_index[1], data.x.size(0))
deg = deg.pow(-0.5)

In [26]:
norm = deg[data.edge_index[0]] * deg[data.edge_index[1]]

In [27]:
norm.shape

torch.Size([10556])